In [15]:
#Ex 1
class Romberg:
    def __init__(self,f,a,b):
        self.f = f
        self.a = a
        self.b = b
        self.p = 0
        self.N = 1
        self.S = 0.5*(f(a)+f(b))
        self.h = (b-a)
        self.I = self.S*self.h
        self.I_last = self.I
        self.n_eval = 0
    def iteration(self):
        self.n_eval += self.N
        x = self.a+self.h*0.5
        somme = self.f(x)
        for k in range(self.N-1):
            x += self.h
            somme += f(x)
        self.N *= 2
        self.p += 1
        self.S += somme
        self.h *= 0.5
        self.I_last = self.I
        self.I = self.h*self.S
        
    def iterations(self,P):
        I = [self.I]
        h = [self.h]
        while self.p<P:
            self.iteration()
            I.append(self.I)
            h.append(self.h)
        return (numpy.array(h),numpy.array(I))
        
    def trapezes(self,epsilon):
        I = [self.I]
        h = [self.h]
        self.iteration()
        while abs(self.I-self.I_last)>epsilon:
            self.iteration()
            I.append(self.I)
            h.append(self.h)
        return (numpy.array(h),numpy.array(I))
        
    def romberg(self,epsilon):
        jmax = 20
        A=numpy.zeros((jmax+1,jmax+1))
        A[0][0] = self.I
        self.iteration()
        A[1][0] = self.I
        correction = (A[1][0]-A[0][0])/3
        A[1][1] = A[1][0] + correction
        j = 2
        while abs(correction) > epsilon:
            self.iteration()
            A[j][0] = self.I
            for i in range(1,j+1):
                correction = (A[j][i-1]-A[j-1][i-1])/(4**i-1)
                A[j][i] = A[j][i-1] + correction
            j += 1
        return (A[0:j-1,0:j-1],A[j-1][j-1])

In [16]:
import numpy
def f(x):
    return x**5
romberg = Romberg(f,0,1)
(h,I) = romberg.trapezes(1e-7)

In [17]:
print(I[-1]-1.0/6)

2.4835268314093994e-08


In [18]:
print(romberg.n_eval)

4095


In [12]:
#Ex 1
import numpy as N
import matplotlib.pyplot as P

import pytest                    # pytest importe pour les tests unitaires

"""
Construction d'un système d'extraction et d'analyse de fichiers de sortie de
dynamique moléculaire afin d'extraire les grandeurs thermodynamiques.
On affichera les ensuite isothermes.
"""

__author__ = "Adrien Licari <adrien.licari@ens-lyon.fr>"


tolerance = 1e-8  # Un seuil de tolérance pour les égalités sur réels


##############################
##### A Simulation class #####
##############################

class Simulation:
    """
    La classe Simulation représente une simulation de dynamique
    moléculaire, donc un point de l'équation d'état. Son constructeur
    doit impérativement être appelé avec le chemin du fichier output
    correspondant. Elle possède des méthodes pour extraire les grandeurs
    thermodynamiques et afficher la run, en pouvant enlever certains pas
    de temps en début de simulation.
    """

    def __init__(self, temp, dens, path):
        """
        Le constructeur doit impérativement être appelé avec le chemin du
        fichier décrivant la simulation, ainsi que ses conditions
        thermodynamiques.
        Args :
                temp,dens(float): La température et la densité de la simulation
                path(string): Le chemin vers le fichier décrivant la simulation
        Raises :
                TypeError si temp ou dens ne sont pas des réels
                IOError si le fichier n'existe pas
        """
        self.temp = float(temp)
        self.dens = float(dens)
        tmp = N.loadtxt(path, skiprows=1).T
        self.pot = tmp[0]
        self.kin = tmp[1]
        self.tot = self.pot + self.kin
        self.press = tmp[2]

    def __str__(self):
        """
        Surcharge de l'opérateur str.
        """
        return "Simulation at {:.0f} g/cc and {:.0f} K ; {:d} timesteps". \
            format(self.dens, self.temp, len(self.pot))

    def thermo(self, skipSteps=0):
        """
        Calcule l'énergie et la pression moyenne au cours de la simulation.
        Renvoie un dictionnaire.
        Args:
                skipSteps(int): Nb de pas à enlever en début de simulation.
        Returns:
                {'T':temperature, 'rho':density,
                 'E':energy, 'P':pressure,
                 'dE':dEnergy, 'dP':dPressure}
        """
        return {'T': self.temp,
                'rho': self.dens,
                'E': self.tot[skipSteps:].mean(),
                'P': self.press[skipSteps:].mean(),
                'dE': self.tot[skipSteps:].std(),
                'dP': self.press[skipSteps:].std()}

    def plot(self, skipSteps=0):
        """
        Affiche l'évolution de la Pression et l'énergie interne au cours de
        la simulation.
        Args:
                skipSteps(int): Pas de temps à enelevr en début de simulation.
        Raises:
                TypeError si skipSteps n'est pas un entier.
        """
        fig, (axen, axpress) = P.subplots(2, sharex=True)
        axen.plot(list(range(skipSteps, len(self.tot))), self.tot[skipSteps:],
                  'rd--')
        axen.set_title("Internal energy (Ha)")
        axpress.plot(list(range(skipSteps, len(self.press))), self.press[skipSteps:],
                     'rd--')
        axpress.set_title("Pressure (GPa)")
        axpress.set_xlabel("Timesteps")

        P.show()

##### Tests pour Simulation #####


def mimic_simulation(filename):
    with open(filename, 'w') as f:
        f.write("""Potential energy (Ha)	Kinetic Energy (Ha)	Pressure (GPa)
-668.2463567264        	0.7755612311   		9287.7370229824
-668.2118514558        	0.7755612311		9286.1395903265
-668.3119088218        	0.7755612311		9247.6604398856
-668.4762735176        	0.7755612311		9191.8574820856
-668.4762735176        	0.7755612311		9191.8574820856
""")


def test_Simulation_init():
    mimic_simulation("equationEtat_simuTest.out")
    s = Simulation(10, 10, "equationEtat_simuTest.out")
    assert len(s.kin) == 5
    assert abs(s.kin[2] - 0.7755612311) < tolerance
    assert abs(s.pot[1] + 668.2118514558) < tolerance


def test_Simulation_str():
    mimic_simulation("equationEtat_simuTest.out")
    s = Simulation(10, 20, "equationEtat_simuTest.out")
    assert str(s) == "Simulation at 20 g/cc and 10 K ; 5 timesteps"


def test_Simulation_thermo():
    mimic_simulation("equationEtat_simuTest.out")
    s = Simulation(10, 20, "equationEtat_simuTest.out")
    assert abs(s.thermo()['T'] - 10) < tolerance
    assert abs(s.thermo()['rho'] - 20) < tolerance
    assert abs(s.thermo()['E'] + 667.56897157674) < tolerance
    assert abs(s.thermo()['P'] - 9241.0504034731) < tolerance
    assert abs(s.thermo(3)['E'] + 667.7007122865) < tolerance
    assert abs(s.thermo(3)['P'] - 9191.8574820856) < tolerance

###################
### Main script ###
###################

if __name__ == '__main__':
    """
    On définit un certain nombre de pas de temps à sauter, puis on
    charge chaque simulation et extrait les informaions thermodynamiques
    associées. On affiche enfin les isothermes normalisées (E/NkT et P/nkT).
    """

    ### Definitions ###
    a0 = 0.52918      # Bohr radius in angstrom
    amu = 1.6605      # atomic mass unit in e-24 g
    k_B = 3.16681e-6  # Boltzmann's constant in Ha/K
    # normalization factor for P/nkT
    nk_GPa = a0 ** 3 * k_B * 2.942e4 / 6 / amu
    nsteps = 200  # define skipped timesteps (should be done for
    # each simulation...)
    temps = [6000, 20000, 50000]    # define temperatures
    colors = {6000: 'r', 20000: 'b', 50000: 'k'}
    denss = [7, 15, 25, 30]  # define densities
    keys = ['T', 'rho', 'E', 'dE', 'P', 'dP']
    eos = dict.fromkeys(keys, N.zeros(0))   # {key:[]}

    ### Extract the EOS out of the source files ###
    for t, rho in [(t, rho) for t in temps for rho in denss]:
        filenm = "outputs/{}K_{:0>2d}gcc.out".format(t, rho)
        s = Simulation(t, rho, filenm)
        for key in keys:
            eos[key] = N.append(eos[key], s.thermo(nsteps)[key])

    ### Plot isotherms ###
    fig, (axen, axpress) = P.subplots(2, sharex=True)
    fig.suptitle("High-pressure equation of state for water", size='x-large')
    axen.set_title("Energy")
    axen.set_ylabel("U / NkT")
    axpress.set_title("Pressure")
    axpress.set_ylabel("P / nkT")
    axpress.set_xlabel("rho (g/cc)")
    for t in temps:
        sel = eos['T'] == t
        axen.errorbar(x=eos['rho'][sel], y=eos['E'][sel] / k_B / t,
                      yerr=eos['dE'][sel] / k_B / t, fmt=colors[t] + '-')
        axpress.errorbar(x=eos['rho'][sel],
                         y=eos['P'][sel] / eos['rho'][sel] / nk_GPa / t,
                         yerr=eos['dP'][sel] / eos['rho'][sel] / nk_GPa / t,
                         fmt=colors[t] + '-',
                         label="{} K".format(t))
    axpress.legend(loc='best')
    P.show()

OSError: outputs/6000K_07gcc.out not found.

In [9]:
#Ex 3
class Ville:
   def __init__(self, lon, lat, nom):
      self.lon = lon
      self.lat = lat
      self.nom = nom
   

   def distance(self, ville):
      distanceX = (ville.lon-self.lon)*40000*math.cos((self.lat+ville.lat)*math.pi/360)/360
      distanceY = (self.lat-ville.lat)*40000/360
      distance = math.sqrt( (distanceX*distanceX) + (distanceY*distanceY) )
      return distance
    
class GestionnaireCircuit:
   villesDestinations = []
   
   def ajouterVille(self, ville):
      self.villesDestinations.append(ville)
   
   def getVille(self, index):
      return self.villesDestinations[index]
   
   def nombreVilles(self):
      return len(self.villesDestinations)

class Circuit:
   def __init__(self, gestionnaireCircuit, circuit=None):
      self.gestionnaireCircuit = gestionnaireCircuit
      self.circuit = []
      self.fitness = 0.0
      self.distance = 0
      if circuit is not None:
         self.circuit = circuit
      else:
         for i in range(0, self.gestionnaireCircuit.nombreVilles()):
            self.circuit.append(None)

   def __len__(self):
      return len(self.circuit)
   
   def __getitem__(self, index):
     return self.circuit[index]

   def __setitem__(self, key, value):
     self.circuit[key] = value

   def genererIndividu(self):
     for indiceVille in range(0, self.gestionnaireCircuit.nombreVilles()):
        self.setVille(indiceVille, self.gestionnaireCircuit.getVille(indiceVille))
     random.shuffle(self.circuit)

   def getVille(self, circuitPosition):
     return self.circuit[circuitPosition]

   def setVille(self, circuitPosition, ville):
     self.circuit[circuitPosition] = ville
     self.fitness = 0.0
     self.distance = 0

   def getFitness(self):
     if self.fitness == 0:
        self.fitness = 1/float(self.getDistance())
     return self.fitness

   def getDistance(self):
     if self.distance == 0:
        circuitDistance = 0
        for indiceVille in range(0, self.tailleCircuit()):
           villeOrigine = self.getVille(indiceVille)
           villeArrivee = None
           if indiceVille+1 < self.tailleCircuit():
              villeArrivee = self.getVille(indiceVille+1)
           else:
              villeArrivee = self.getVille(0)
           circuitDistance += villeOrigine.distance(villeArrivee)
        self.distance = circuitDistance
     return self.distance

   def tailleCircuit(self):
     return len(self.circuit)

   def contientVille(self, ville):
     return ville in self.circuit

class Population:
   def __init__(self, gestionnaireCircuit, taillePopulation, init):
      self.circuits = []
      for i in range(0, taillePopulation):
         self.circuits.append(None)
      
      if init:
         for i in range(0, taillePopulation):
            nouveauCircuit = Circuit(gestionnaireCircuit)
            nouveauCircuit.genererIndividu()
            self.sauvegarderCircuit(i, nouveauCircuit)
      
   def __setitem__(self, key, value):
      self.circuits[key] = value
   
   def __getitem__(self, index):
      return self.circuits[index]
   
   def sauvegarderCircuit(self, index, circuit):
      self.circuits[index] = circuit
   
   def getCircuit(self, index):
      return self.circuits[index]
   
   def getFittest(self):
      fittest = self.circuits[0]
      for i in range(0, self.taillePopulation()):
         if fittest.getFitness() <= self.getCircuit(i).getFitness():
            fittest = self.getCircuit(i)
      return fittest
   
   def taillePopulation(self):
      return len(self.circuits)
    
class GA:
   def __init__(self, gestionnaireCircuit):
      self.gestionnaireCircuit = gestionnaireCircuit
      self.tauxMutation = 0.015
      self.tailleTournoi = 5
      self.elitisme = True
   
      def evoluerPopulation(self, pop):
      nouvellePopulation = Population(self.gestionnaireCircuit, pop.taillePopulation(), False)
      elitismeOffset = 0
      if self.elitisme:
         nouvellePopulation.sauvegarderCircuit(0, pop.getFittest())
         elitismeOffset = 1
      
      for i in range(elitismeOffset, nouvellePopulation.taillePopulation()):
         parent1 = self.selectionTournoi(pop)
         parent2 = self.selectionTournoi(pop)
         enfant = self.crossover(parent1, parent2)
         nouvellePopulation.sauvegarderCircuit(i, enfant)
      
      for i in range(elitismeOffset, nouvellePopulation.taillePopulation()):
         self.muter(nouvellePopulation.getCircuit(i))
      
      return nouvellePopulation


    def crossover(self, parent1, parent2):
      enfant = Circuit(self.gestionnaireCircuit)
      
      startPos = int(random.random() * parent1.tailleCircuit())
      endPos = int(random.random() * parent1.tailleCircuit())
      
      for i in range(0, enfant.tailleCircuit()):
         if startPos < endPos and i > startPos and i < endPos:
            enfant.setVille(i, parent1.getVille(i))
         elif startPos > endPos:
            if not (i < startPos and i > endPos):
               enfant.setVille(i, parent1.getVille(i))
      
      for i in range(0, parent2.tailleCircuit()):
         if not enfant.contientVille(parent2.getVille(i)):
            for ii in range(0, enfant.tailleCircuit()):
               if enfant.getVille(ii) == None:
                  enfant.setVille(ii, parent2.getVille(i))
                  break
      
      return enfant
   
   def muter(self, circuit):
     for circuitPos1 in range(0, circuit.tailleCircuit()):
        if random.random() < self.tauxMutation:
           circuitPos2 = int(circuit.tailleCircuit() * random.random())
           
           ville1 = circuit.getVille(circuitPos1)
           ville2 = circuit.getVille(circuitPos2)
           
           circuit.setVille(circuitPos2, ville1)
           circuit.setVille(circuitPos1, ville2)

   def selectionTournoi(self, pop):
     tournoi = Population(self.gestionnaireCircuit, self.tailleTournoi, False)
     for i in range(0, self.tailleTournoi):
        randomId = int(random.random() * pop.taillePopulation())
        tournoi.sauvegarderCircuit(i, pop.getCircuit(randomId))
     fittest = tournoi.getFittest()
     return fittest
    
    if __name__ == '__main__':
   
   gc = GestionnaireCircuit()   

   #on cree nos villes
   ville1 = Ville(3.002556, 45.846117, 'Clermont-Ferrand')
   gc.ajouterVille(ville1)
   ville2 = Ville(-0.644905, 44.896839, 'Bordeaux')
   gc.ajouterVille(ville2)
   ville3 = Ville(-1.380989, 43.470961, 'Bayonne')
   gc.ajouterVille(ville3)
   ville4 = Ville(1.376579, 43.662010, 'Toulouse')
   gc.ajouterVille(ville4)
   ville5 = Ville(5.337151, 43.327276, 'Marseille')
   gc.ajouterVille(ville5)
   ville6 = Ville(7.265252, 43.745404, 'Nice')
   gc.ajouterVille(ville6)
   ville7 = Ville(-1.650154, 47.385427, 'Nantes')
   gc.ajouterVille(ville7)
   ville8 = Ville(-1.430427, 48.197310, 'Rennes')
   gc.ajouterVille(ville8)
   ville9 = Ville(2.414787, 48.953260, 'Paris')
   gc.ajouterVille(ville9)
   ville10 = Ville(3.090447, 50.612962, 'Lille')
   gc.ajouterVille(ville10)
   ville11 = Ville(5.013054, 47.370547, 'Dijon')
   gc.ajouterVille(ville11)
   ville12 = Ville(4.793327, 44.990153, 'Valence')
   gc.ajouterVille(ville12)
   ville13 = Ville(2.447746, 44.966838, 'Aurillac')
   gc.ajouterVille(ville13)
   ville14 = Ville(1.750115, 47.980822, 'Orleans')
   gc.ajouterVille(ville14)
   ville15 = Ville(4.134148, 49.323421, 'Reims')
   gc.ajouterVille(ville15)
   ville16 = Ville(7.506950, 48.580332, 'Strasbourg')
   gc.ajouterVille(ville16)
   ville17 = Ville(1.233757, 45.865246, 'Limoges')
   gc.ajouterVille(ville17)
   ville18 = Ville(4.047255,48.370925, 'Troyes')
   gc.ajouterVille(ville18)
   ville19 = Ville(0.103163,49.532415, 'Le Havre')
   gc.ajouterVille(ville19)
   ville20 = Ville(-1.495348, 49.667704, 'Cherbourg')
   gc.ajouterVille(ville20)
   ville21 = Ville(-4.494615, 48.447500, 'Brest')
   gc.ajouterVille(ville21)
   ville22 = Ville(-0.457140, 46.373545, 'Niort')
   gc.ajouterVille(ville22)


   #on initialise la population avec 50 circuits
   pop = Population(gc, 50, True)
   print "Distance initiale : " + str(pop.getFittest().getDistance())
   
   # On fait evoluer notre population sur 100 generations
   ga = GA(gc)
   pop = ga.evoluerPopulation(pop)
   for i in range(0, 100):
      pop = ga.evoluerPopulation(pop)
   
   print "Distance finale : " + str(pop.getFittest().getDistance())
   meilleurePopulation = pop.getFittest()

   #on genere une carte représentant notre solution
   lons = []
   lats = []
   noms = []
   for ville in meilleurePopulation.circuit:
      lons.append(ville.lon)
      lats.append(ville.lat)
      noms.append(ville.nom)

   lons.append(lons[0])
   lats.append(lats[0])
   noms.append(noms[0])

   map = Basemap(llcrnrlon=-5.5,llcrnrlat=42.3,urcrnrlon=9.3,urcrnrlat=51.,
             resolution='i', projection='tmerc', lat_0 = 45.5, lon_0 = -3.25)

   map.drawmapboundary(fill_color='aqua')
   map.fillcontinents(color='coral',lake_color='aqua')
   map.drawcoastlines()
   map.drawcountries()
   x,y = map(lons,lats)
   map.plot(x,y,'bo', markersize=12)
   for nom,xpt,ypt in zip(noms,x,y):
       plt.text(xpt+5000,ypt+25000,nom)

   map.plot(x, y, 'D-', markersize=10, linewidth=2, color='k', markerfacecolor='b') 
   plt.show()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 146)